In [1]:
import pandas as pd

In [2]:
import json
video_caps = open('/l/users/xinyue.li/caption/SwinBERT/output/results.json')
video_caps = json.load(video_caps)

In [3]:
audio_names_test = list(video_caps.keys())

### inference

In [2]:
import torch.cuda
import llama
from util.misc import *
from data.utils import load_and_transform_audio_data

/home/fathinah.izzati/miniconda3/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/fathinah.izzati/miniconda3/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
print('loading llama')
model = './ckpts/checkpoint.pth'
llama_dir = '/l/users/fathinah.izzati/ml711/llama'
mert_path = 'm-a-p/MERT-v1-330M'
knn_dir = './ckpts'
llama_type = 'llama-2-7b-chat'
model = llama.load(model_path=model, llama_dir=llama_dir, mert_path=mert_path, knn=True, knn_dir=knn_dir, llama_type=llama_type)
print('Model LLAMA is loaded')
model.eval()
print('Model LLAMA is eval()')

loading llama
Loading LLaMA-Adapter from ./ckpts/checkpoint.pth


Some weights of the model checkpoint at m-a-p/MERT-v1-330M were not used when initializing MERTModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing MERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MERTModel were not initialized from the model checkpoint at m-a-p/MERT-v1-330M and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model args: ModelArgs(dim=4096, n_layers=32, n_heads=32, n_kv_heads=None, vocab_size=-1, multiple_of=256, ffn_dim_multiplier=None, norm_eps=1e-06, max_batch_size=1, max_seq_len=8192, w_bias=True, w_lora=True, lora_rank=16)


/home/fathinah.izzati/miniconda3/lib/python3.11/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1711403378171/work/torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [ ]:
def multimodal_generate(
        audio_path,
        audio_weight,
        prompts,
        cache_size,
        cache_t,
        cache_weight,
        max_gen_len,
        gen_t, top_p
):
    inputs = {}
    audio = load_and_transform_audio_data([audio_path])
    inputs['Audio'] = [audio, audio_weight]
    text_output = None
    with torch.cuda.amp.autocast():
        results = model.generate(inputs, prompts, max_gen_len=max_gen_len, temperature=gen_t, top_p=top_p,
                                     cache_size=cache_size, cache_t=cache_t, cache_weight=cache_weight)
    text_output = results[0].strip()
    return text_output

In [ ]:
def encode_prompt(prompt):
    prompts = [llama.format_prompt(prompt)]
    prompts = [model.tokenizer.encode(x, bos=True, eos=False) for x in prompts]
    return prompts
# q = 'Describe the audio with at least 3 adjectives'
q1 = 'Describe elements of the music, such as its mood, rhythm, and genre'
prompts1 = encode_prompt(q1)
q2 = 'Describe musical features from this audio in detail'
prompts2 = encode_prompt(q2)
q3 = 'Describe what you hear'
prompts3 = encode_prompt(q3)

In [ ]:
for audio in audio_names_test:
    audio_path = f'/l/users/xinyue.li/caption/dataset/audios/{audio}.mp3'
    print(audio_path)
    print('Start generate')
    output1 = multimodal_generate(audio_path, 1, prompts1, 100, 20.0, 0.0, 512, 0.6, 0.8)
    output2 = multimodal_generate(audio_path, 1, prompts2, 100, 20.0, 0.0, 512, 0.6, 0.8)
    output3 = multimodal_generate(audio_path, 1, prompts3, 100, 20.0, 0.0, 512, 0.6, 0.8)
    print('Success')
    print(f"Audio File: {audio_path}")
    print(f"Q: {q}")
    print(f"A: {output}")
    outputs1 = {}
    outputs2 = {}
    outputs3 = {}
    outputs1[audio]={'pred':output1, 'q':q}
    outputs2[audio]={'pred':output2, 'q':q}
    outputs3[audio]={'pred':output3, 'q':q}
    a =  open('mullama_output1.json')
    b =  open('mullama_output2.json')
    c =  open('mullama_output3.json')
    z1 = json.load(a)
    z2 = json.load(b)
    z3 = json.load(c)
    z1.update(outputs1)
    z2.update(outputs2)
    z3.update(outputs3)
    with open('mullama_output1.json', 'w', encoding ='utf8') as x: 
        json.dump(z1, x)
    with open('mullama_output2.json', 'w', encoding ='utf8') as x: 
        json.dump(z2, x)
    with open('mullama_output3.json', 'w', encoding ='utf8') as x: 
        json.dump(z3, x)

In [ ]:
# audio_caps = open('mullama_output.json')
# audio_caps = json.load(audio_caps)